In [164]:
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
import numpy as np
import tqdm
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [165]:
raw_sop = pd.read_csv('./data/sops.csv')

In [166]:
raw_sop.head()

,city,sop_doc
0,Austin,Austin Police DepartmentPolicy ManualCopyright...
1,Austin,Austin Police DepartmentPolicy ManualCopyright...
2,Austin,Austin Police DepartmentPolicy ManualCopyright...
3,Austin,Austin Police DepartmentPolicy ManualCanons of...
4,Austin,Austin Police DepartmentPolicy ManualCanons of...


In [167]:
sop = raw_sop.drop(columns=['city'])

In [168]:
sop.sample(10)

,sop_doc
1082,Dallas Police Department General Order \n 601....
969,Dallas Police Department General Order \n 414...
913,\nDallas Police Depa\nrtment General Order\n ...
2222,Policy\n331\nEvanston Police Department\nPolic...
549,Austin Police DepartmentPolicy ManualAdministr...
1977,Interview room videos saved in Exacq will show...
6105,Las Vegas Metropolitan Police DepartmentPartne...
4170,UNIFORM POLICY\n Operations Order\n 3.15\n PHO...
3932,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5104,Professional Conduct and Ethics\n - RECEIVED ...


In [169]:
sop['sop_doc_processed'] = sop[
    'sop_doc'
].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<ipython-input-169-da76bbe04fda>:3: DeprecationWarning: invalid escape sequence \.
  ].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))


In [170]:
sop['sop_doc_processed'] = sop['sop_doc_processed'].map(lambda x: x.lower().strip())

In [171]:
sop = sop[sop['sop_doc_processed'] != '']

In [172]:
sop.sample(10)

,sop_doc,sop_doc_processed
2384,Policy\n425\nEvanston Police Department\nPolic...,policy 425 evanston police department policy m...
1546,September\n 2015 333 The investigating office...,september 2015 333 the investigating officer...
748,Policy1024Austin Police DepartmentPolicy Manua...,policy1024austin police departmentpolicy manua...
535,Austin Police DepartmentPolicy ManualAdministr...,austin police departmentpolicy manualadministr...
4003,TRANSFER POLICY\n Operations Order\n 3.2\n PHO...,transfer policy operations order 32 phoenix...
3872,USE OF FORCE\n Operations Order\n 1.5\n PHOENI...,use of force operations order 15 phoenix po...
109,Austin Police DepartmentPolicy ManualVehicle P...,austin police departmentpolicy manualvehicle p...
5812,Index\n Wearing of helment & gas mask\n ........,index wearing of helment & gas mask 686 we...
822,\nDallas Police Department General Order\n \n...,dallas police department general order 300 0...
6116,Las Vegas Metropolitan Police DepartmentPartne...,las vegas metropolitan police departmentpartne...


In [184]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [185]:
data = sop.sop_doc_processed.values.tolist()
data_words = np.array(list(sent_to_words(data)))

['austin',
 'police',
 'manualresponse',
 'to',
 'lexipol',
 'llc',
 'all',
 'rights',
 'with',
 'permission',
 'by',
 'austin',
 'police',
 'to',
 'resistance',
 'de',
 'escalation',
 'using',
 'techniques',
 'to',
 'stabilize',
 'the',
 'situation',
 'and',
 'reduce',
 'the',
 'immediacy',
 'of',
 'the',
 'threat',
 'so',
 'that',
 'more',
 'options',
 'and',
 'resources',
 'are',
 'available',
 'to',
 'bring',
 'about',
 'successful',
 'resolution',
 'to',
 'an',
 'encounter',
 'with',
 'non',
 'compliant',
 'subject',
 'the',
 'goal',
 'of',
 'de',
 'escalation',
 'is',
 'to',
 'gain',
 'the',
 'voluntary',
 'compliance',
 'of',
 'subjects',
 'when',
 'feasible',
 'thereby',
 'eliminating',
 'the',
 'need',
 'to',
 'use',
 'force',
 'in',
 'response',
 'to',
 'resistancede',
 'escalation',
 'techniques',
 'communicative',
 'or',
 'other',
 'actions',
 'used',
 'by',
 'officers',
 'when',
 'safe',
 'and',
 'without',
 'compromising',
 'law',
 'enforcement',
 'priorities',
 'that',
 